In [4]:


## STEP 1: Set up the workspace.

# import json
import os
from dotenv import load_dotenv
load_dotenv()

my_secret = os.getenv("MY_SECRET")

#GETTING MY ID:
#id = open('id_CONFIDENTIAL.json', 'r')
#mi = json.load(id)
my_id = my_secret

from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = my_id,
                      resource_group = "clasemlcloud",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)



Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-e03bdb9407
Deploying KeyVault with name workspackeyvault75d13b3a.
Deploying StorageAccount with name workspacstoragef3dbd11f8.
Deploying AppInsights with name workspacinsights0023385f.
Deployed AppInsights with name workspacinsights0023385f. Took 55.32 seconds.
Deploying Workspace with name workspace.
Deployed Workspace with name workspace. Took 43.96 seconds.
Registering model model


In [ ]:


################ score.py ###################

umb = open("umbral.json", "r")
umb = json.load(umb)
umbral = umb["umbral"][0]

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)

def sigmoid(x):
  return [1 / (1 + np.exp(-y)) for y in x]

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    data_dummies = data.drop(["RowNumber", "CustomerId", "Surname", "Geography", "Gender", "Card Type"], axis=1) ## INCLUIR AQUÍ LO DE ZORROUNO

    result = model.predict(data_dummies).tolist()
    result_sigmoid = sigmoid(result)
    umbral = {umbral}
    result_finals = [1 if x > umbral else 0 for x in result_sigmoid]

    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

